<a href="https://colab.research.google.com/github/ComputerGallI/FinalProjRepo/blob/main/FinalProjectMAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Hugging Face Transformers
!pip install transformers datasets sentence-transformers

# Import dependencies
from transformers import pipeline
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install transformers datasets sentence-transformers faiss-cpu --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.4 MB/s eta 0:00:00


In [7]:
# Clone the Google Research repo that contains go_emotions
!git clone https://github.com/google-research/google-research.git
!pip install pandas


Cloning into 'google-research'...
remote: Enumerating objects: 106502, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 106502 (delta 10), reused 6 (delta 5), pack-reused 106478 (from 2)
Receiving objects: 100% (106502/106502), 1.09 GiB | 25.08 MiB/s, done.
Resolving deltas: 100% (69409/69409), done.
Updating files: 100% (22361/22361), done.


In [9]:
import pandas as pd

# Load train data from the local repo
df = pd.read_csv("google-research/goemotions/data/train.tsv", sep="\t", header=None)

# Optional: peek at the top rows
df.head()


,0,1,2
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


In [10]:
texts = df[0].tolist()[:1000]


In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load a lightweight BERT retriever model
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode your texts into embeddings
text_embeddings = retriever_model.encode(texts, convert_to_numpy=True)

# Build a FAISS index for fast retrieval
dimension = text_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(text_embeddings)

print("✅ FAISS index built successfully with", len(texts), "documents.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS index built successfully with 1000 documents.


In [12]:
from transformers import pipeline

# Load a text generation pipeline using BART
generator = pipeline("text2text-generation", model="facebook/bart-large")

def rag_response(user_query, top_k=3):
    # Step 1: Embed user query
    query_vector = retriever_model.encode([user_query])

    # Step 2: Retrieve top_k similar texts from FAISS
    distances, indices = index.search(np.array(query_vector), top_k)
    retrieved_docs = [texts[i] for i in indices[0]]

    # Step 3: Prepare the input prompt for BART
    context = " ".join(retrieved_docs)
    prompt = f"Context: {context}\nQuestion: {user_query}"

    # Step 4: Generate response using BART
    response = generator(prompt, max_length=100, do_sample=False)
    return response[0]['generated_text']


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [13]:
print(rag_response("I feel like I can’t focus today. What should I do?"))


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Context: Try to keep yourself busy with other things to forget the thoughts. Focus on yourself by focusing on your prayers and reminding [NAME]Question: I feel like I can’t focus today. I focus on the wrong things, but I’m not sure why. What should I do?
